In [1]:
import pandas as pd

df = pd.read_csv('../data/csv/data.csv')

print(df.shape)
df.head()

(1254243, 6)


,CLNT_RMT_IP,ROI_CLICK_EVENT_TS,BRWSR_NAME,ams_pblshr_id,buyer_id,click_id
0,77.111.247.168,2020-08-02 18:37:01,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5574672411,valen_949,209245484389035777
1,66.249.73.101,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5574672411,morbanisaf,209453736892211586
2,77.111.247.129,2020-08-21 03:36:38,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5574672411,morbanisaf,209453736892211586
3,66.249.73.101,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5574672411,morbanisaf,209453736892211586
4,77.111.247.129,2020-08-21 03:36:33,Mozilla/5.0 (Windows NT 10.0; Win64; x64) Appl...,5574672411,morbanisaf,209453736892211586


In [ ]:
from user_agents import parse
import pandas as pd
import swifter  # for parallel processing with Pandas (install with `pip install swifter`)

# Dictionary to cache parsed user-agent strings and reduce redundant parsing
user_agent_cache = {}

def extract_user_agent_features_optimized(ua_string):
    # Check if this user-agent string has already been parsed
    if ua_string in user_agent_cache:
        return user_agent_cache[ua_string]
    
    # Parse the user agent string and store parsed results in a variable
    user_agent = parse(ua_string)
    
    # Extract required features
    result = {
        'browser': user_agent.browser.family,
        # 'browser_version': user_agent.browser.version_string,
        'os': user_agent.os.family,
        # 'os_version': user_agent.os.version_string,
        'device_type': (
    'mobile' if user_agent.is_mobile else
    'desktop' if user_agent.is_pc else
    'tablet' if user_agent.is_tablet else
    'bot' if user_agent.is_bot else 'other'
        )
    }
    
    # Store result in the cache
    user_agent_cache[ua_string] = result
    
    return result

# Apply optimized function to the user-agent column with parallel processing using swifter
df_user_agent_features = df['BRWSR_NAME'].swifter.apply(extract_user_agent_features_optimized).apply(pd.Series)
df = pd.concat([df, df_user_agent_features], axis=1)
df = df.drop('BRWSR_NAME', axis=1)
def categorize_browser(browser_column):
    # Define the browsers to keep based on the 10,000 row threshold
    popular_browsers = ['Chrome Mobile', 'Chrome', 'Mobile Safari', 'Mobile Safari UI/WKWebView', 'Edge', 'Firefox', 'Samsung Internet', 'Safari', 'Google', 'Chrome Mobile', 'Opera', 'FacebookBot', 'Facebook', 'ImagesiftBot']

    # Apply the categorization
    browser_column = browser_column.apply(lambda x: x if x in popular_browsers else 'Other')
    return browser_column

# Function to categorize OS based on the 1% rule and keeping till Ubuntu
def categorize_os(os_column):
    # Define the OSes to keep based on the row threshold (up to Ubuntu)
    popular_os = ['Android', 'Other', 'iOS', 'Windows', 'Mac OS X', 'Linux', 'Ubuntu', 'Chrome OS']

    # Apply the categorization
    os_column = os_column.apply(lambda x: x if x in popular_os else 'Other')
    return os_column

# Apply categorization functions to the DataFrame
df['browser'] = categorize_browser(df['browser'])
df['os'] = categorize_os(df['os'])

# Display the transformed DataFrame
print(df)

Pandas Apply:   0%|          | 0/1254243 [00:00<?, ?it/s]

In [ ]:
import re
import socket
import struct
from sklearn.preprocessing import OneHotEncoder


# Function to check if the IP address is valid
def is_valid_ipv4(ip):
    if pd.isna(ip):  # Check for None or NaN
        return False
    pattern = r'^(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)\.(25[0-5]|2[0-4][0-9]|[01]?[0-9][0-9]?)$'
    return re.match(pattern, ip) is not None

# Apply the function and create a mask for valid IP addresses
valid_ips_mask = df['CLNT_RMT_IP'].apply(is_valid_ipv4)

# Drop rows with invalid IP addresses
df = df[valid_ips_mask]



# 1. Convert IP to integer
def ipv4_to_int(ip):
    return struct.unpack("!I", socket.inet_aton(ip))[0]

df['ip_as_int'] = df['CLNT_RMT_IP'].apply(ipv4_to_int)
df['ROI_CLICK_EVENT_TS'] = pd.to_datetime(df['ROI_CLICK_EVENT_TS'])


# 3. Extract features from datetime
df['year'] = df['ROI_CLICK_EVENT_TS'].dt.year
df['month'] = df['ROI_CLICK_EVENT_TS'].dt.month
df['day'] = df['ROI_CLICK_EVENT_TS'].dt.day
df['hour'] = df['ROI_CLICK_EVENT_TS'].dt.hour
df['minute'] = df['ROI_CLICK_EVENT_TS'].dt.minute
df['second'] = df['ROI_CLICK_EVENT_TS'].dt.second
df['day_of_week'] = df['ROI_CLICK_EVENT_TS'].dt.dayofweek  # Monday=0, Sunday=6

In [ ]:
import pandas as pd
from itertools import combinations
from sklearn.preprocessing import StandardScaler



# Step 1: Generate all possible combinations of 'CLNT_RMT_IP', 'ams_pblshr_id', 'buyer_id'
columns = ['CLNT_RMT_IP', 'ams_pblshr_id', 'buyer_id']
combinations_list = []

# Add single column combinations
for i in range(1, len(columns) + 1):
    combinations_list.extend(combinations(columns, i))

# Step 2: Group by each combination and get the count of 'click_id'
result_list = []
for comb in combinations_list:
    grouped = df.groupby(list(comb)).size().reset_index(name=f'click_count_{"+".join(comb)}')
    result_list.append(grouped)
print(len(result_list))

# Step 3: Merge all grouped results back into the original dataframe
merged_df = df.copy()
for result in result_list:
    merged_df = pd.merge(merged_df, result, how='left', on=list(result.columns[:-1]))  # Exclude 'click_count'

# Step 4: Handle missing values (if any) and fill them with 0 or another appropriate value
merged_df.fillna(0, inplace=True)

click_count_columns = [col for col in merged_df.columns if col.startswith('click_count')]

# Step 6: Apply scaling to the 'click_count' columns
scaler = StandardScaler()
merged_df[click_count_columns] = scaler.fit_transform(merged_df[click_count_columns])

# Output the final DataFrame with scaled features
print("DataFrame with Scaled Click Count Features:")
print(merged_df)
